In [1]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import torch
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 08-23 19:13:17 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-23 19:13:17 [__init__.py:239] Automatically detected platform cuda.


In [3]:
max_seq_length = 1024  # Can increase for longer reasoning traces
lora_rank = 32         # Larger rank = smarter, but slower

# Load model + tokenizer with vLLM acceleration
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-1.7B",
    max_seq_length = max_seq_length,
    load_in_4bit = True,       # False for LoRA 16bit
    fast_inference = True,      # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

Unsloth: Patching vLLM v1 graph capture
Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.5.7: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Quadro T1000 with Max-Q Design. Num GPUs = 1. Max memory: 4.0 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/qwen3-1.7b-unsloth-bnb-4bit with actual GPU utilization = 55.83%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 4.0 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 1024. Num Sequences = 128.
Unsloth: vLLM's KV Cache can use up to 0.77 GB. Also swap space = 0 GB.
WARNING 08-23 19:13:25 [config.py:2972] Casting torch.bfloat16 to torch.float16.
INFO 08-23 19:13:

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 08-23 19:13:40 [punica_selector.py:18] Using PunicaWrapperGPU.
INFO 08-23 19:13:40 [model_runner.py:1140] Model loading took 1.3985 GiB and 3.064188 seconds
INFO 08-23 19:13:54 [worker.py:287] Memory profiling takes 13.24 seconds
INFO 08-23 19:13:54 [worker.py:287] the current vLLM instance can use total_gpu_memory (4.00GiB) x gpu_memory_utilization (0.56) = 2.23GiB
INFO 08-23 19:13:54 [worker.py:287] model weights take 1.40GiB; non_torch_memory takes 0.02GiB; PyTorch activation peak memory takes 0.70GiB; the rest of the memory reserved for KV Cache is 0.11GiB.
INFO 08-23 19:13:54 [executor_base.py:112] # cuda blocks: 66, # CPU blocks: 0
INFO 08-23 19:13:54 [executor_base.py:117] Maximum concurrency for 1024 tokens per request: 1.03x
INFO 08-23 19:13:54 [vllm_utils.py:671] Unsloth: Running patched vLLM v0 `capture_model`.
INFO 08-23 19:13:54 [model_runner.py:1450] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the mod

Capturing CUDA graph shapes:   0%|          | 0/19 [00:00<?, ?it/s]

INFO 08-23 19:14:28 [model_runner.py:1592] Graph capturing finished in 34 secs, took 0.40 GiB
INFO 08-23 19:14:28 [vllm_utils.py:678] Unsloth: Patched vLLM v0 graph capture finished in 34 secs.
INFO 08-23 19:14:28 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 47.91 seconds
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']
Unsloth: Just some info: will skip parsing ['pre_feedforward_layernorm', 'post_feedforward_layernorm']


In [4]:
from src.globals import TRAINING_ROWS, TRAINING_COLS

In [5]:
REASONING_START = "<think>"
REASONING_END   = "</think>"
SOLUTION_START  = "<SOLUTION>"
SOLUTION_END    = "</SOLUTION>"

SYSTEM_PROMPT = f"""You are a Minesweeper assistant.
The game board is always {TRAINING_ROWS}x{TRAINING_COLS} in size.
You will be given ONLY the current board state as input from the user.

Your task: Suggest exactly ONE valid next move for the minesweeper board given by the user.

Move format rules (must follow exactly one of these two):
1. "row: NUM, col: NUM, action: reveal"       → to reveal a cell
2. "row: NUM, col: NUM, action: flag"         → to flag a cell as a mine

Board representation:
- '*' means the tile has not been revealed yet.
- Numbers 0–8 show how many mines are adjacent to that square.
- 'F' means the tile has already been flagged as a mine.
- The board will be displayed as a grid of symbols only.

Important condition:
- You may only suggest moves on cells that contain '*'.  
- Do NOT suggest moves on numbers or flagged tiles, as these have already been revealed or correctly flagged.

Summary:
- Suggest one valid move next with the format "row: NUM, col: NUM, action: reveal" or "row: NUM, col: NUM, action: flag", where NUM is an integer in the range [1, {TRAINING_ROWS}] for rows and [1, {TRAINING_COLS}] for columns.
- Do NOT explain your reasoning or thought process. Only output the valid move.
"""
# SYSTEM_PROMPT = f"""You are a helpful minesweeper assistant where the goal of the game is to reveal all tiles on the board which are not mines. If you reveal a mine, you lose the game. The user is going to provide you a {TRAINING_ROWS}x{TRAINING_COLS} minesweeper board representation where '*' means unrevealed tile, 'F' means a correctly flagged mine, and numbers 0-8 indicates the tile is revealed and it has that many mines are adjacent to it. Your task is to suggest exactly one valid next move in the format "row col" to reveal a tile or "row col f" to flag a tile as a mine. You may only suggest moves on tiles that contain '*'. Do NOT suggest moves on numbers or flagged tiles. Row and column values should be integers in the range [1, {TRAINING_ROWS}] and [1, {TRAINING_COLS}] respectively. Provide reasoning and then output the one valid move you decide on in the next line. Do NOT repeat or copy the board in your output. Here is the user board:"""

# SYSTEM_PROMPT = f"""You are going to be given a minesweeper board, and your task is to suggest the next move based on the current state of the board. The board will be represented as a grid of characters, and you need to analyze the grid to determine the best move. Here is the user board:"""


In [6]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
#     {"role": "user", "content": """
# * * * * * *
# * * F * * *
# * * 3 3 4 *
# 1 1 1 0 3 *
# 0 0 0 0 3 *
# 0 0 0 0 2 *
# """},
#     {"role": "assistant", "content": "row: 2, col: 4, action: flag"},
#     {"role": "user", "content": """
# 0 1 F 1 2 F
# 0 1 1 1 2 F
# 0 0 0 0 2 2
# 1 2 1 1 1 F
# * * F 2 3 3
# * * * * F F
# """},
#     {"role": "assistant", "content": "row: 6, col: 3, action: reveal"},
    {"role": "user", "content": """
0 0 0 0 0 0
2 2 1 0 1 1
F F 1 0 2 F
3 3 2 0 3 F
1 F 2 1 4 F
* * * * * *
"""}
]

# Apply chat template, enabling thinking mode
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False,
)

# Tokenize input
inputs = tokenizer([text], return_tensors="pt").to(model.device)

# Generate
generated_ids = model.generate(
    **inputs,
    max_new_tokens=512,
)

# Extract only the new tokens
output_ids = generated_ids[0][len(inputs.input_ids[0]):].tolist()
decoded_output = tokenizer.decode(output_ids, skip_special_tokens=True).strip("\n")
print("output:", decoded_output)

# # Extract solution block
# import re
# match = re.search(r"<SOLUTION>(.*?)</SOLUTION>", decoded_output, re.DOTALL)
# if match:
#     move = match.group(1).strip()
# else:
#     move = decoded_output  # fallback if tags missing

# print("final move:", move)



output: row: 1, col: 5, action: reveal
